# Best Practices - Missing Value Imputation

This notebook walks through recommended missing value imputation techniques that can provide smarter imputation than simple mean imputation. Note: for supervised learning models, we recommend leaving missing values as is.  The H2O-3 supervised learning algorithms are able to handle missing values automatically.  

H2O-3 offers two methods for missing value imputation: *GLRM* and *Autoencoder*.  Both algorithms find a low dimensional representation of the data.  With this low dimensional representation, they are able to impute any missing values.  

**GLRM**

* an extension of PCA
* finds linear low-dimensional representation of the data

**Autoencoder**

* a neural network model
* finds non-linear low-dimensional representation of the data

**Steps**

1. Import data
2. Imputation through GLRM model
3. Imputation through Autoencoder

## Step 1. Import Data

For our example, we will use data on [Subject 01's walking stances](https://simtk.org/project/xml/downloads.xml?group_id=603) from an experiment carried out by Hamner and Delp (2013). Each of the 151 rows of the data set contains the (x, y, z) coordinates of major body parts recorded at a specific point in time.

In [26]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,12 mins 52 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_megankurka_vya604
H2O cluster total nodes:,1
H2O cluster free memory:,3.312 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [27]:
gait_data = h2o.import_file(path = "../../tutorials/data/subject01_walk1.csv", destination_frame = "gait.hex")
gait_data.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
0,617.248,1055.28,170.782,639.606,1044.26,-88.9098,430.87,1051.26,29.9668,517.333,741.096,212.083,570.201,666.706,123.686,480.064,626.515,199.554,659.579,745.058,-145.972,750.481,735.85,-73.4851,697.43,642.682,-137.37,360.904,474.04,186.849,420.867,412.366,134.719,343.737,388.037,189.832,750.326,446.177,-128.528,819.764,415.132,-81.7,750.046,397.941,-138.884,156.443,113.073,104.003,297.181,88.1221,67.0685,288.247,73.7662,176.63,429.093,43.0861,110.598,815.764,63.3979,-51.7587,967.99,122.581,-32.7704,943.789,105.337,-122.344,1078.71,143.548,-60.9359,660.827,1353.78,20.1984,566.598,1449.58,229.997,596.424,1468.97,-186.7,593.641,1249.11,279.776,580.735,1233.74,-222.927,567.003,1143.11,315.566,518.103,1146.32,-260.958,771.563,973.779,273.788,746.207,944.378,327.453,619.823,916.17,-218.327,574.163,905.877,-276.121,366.123,35.9082,193.141,375.866,39.8688,81.1719,1022.04,113.138,-136.458,1034.97,125.079,-34.9382,736.522,1571.58,106.421,748.242,1574.96,-31.6672,749.332,1747.61,46.7415
0.017,617.998,1053.22,168.513,641.236,1042.28,-90.9321,432.341,1050.24,26.8468,516.614,740.426,211.219,570.519,664.199,123.394,479.64,625.653,199.315,661.613,743.567,-146.439,756.852,736.423,-77.1608,700.72,641.493,-135.753,357.694,474.357,187.046,415.392,411.818,134.787,336.911,390.057,190.88,755.121,443.931,-128.735,820.42,409.886,-79.0317,750.565,396.968,-137.308,141.984,121.291,105.038,281.979,91.6346,68.4662,271.807,75.7492,177.511,409.884,43.1392,110.564,799.82,65.8006,-49.9463,957.452,113.746,-27.6771,943.789,105.337,-122.344,1069.28,128.481,-56.2015,661.329,1352.71,17.2436,566.925,1448.94,226.133,596.42,1466.41,-191.477,593.205,1248.56,276.044,582.396,1230.64,-225.945,566.137,1142.56,312.591,519.93,1142.17,-262.842,767.166,969.574,272.553,742.406,941.011,325.924,623.055,912.317,-220.419,576.286,902.639,-278.464,346.245,35.811,193.415,358.95,39.8057,80.9351,1011.08,101.084,-132.527,1024,111.614,-30.6797,736.042,1570.83,101.786,748.412,1574.04,-36.6651,750.68,1746.67,41.1642
0.033,620.292,1051.77,165.859,643.597,1041.06,-94.3072,434.099,1049.34,23.8194,517.779,739.681,209.93,570.455,663.091,123.094,479.471,625.037,199.233,663.777,742.559,-147.207,760.839,737.405,-80.5061,702.586,640.568,-134.511,355.966,476.074,187.729,411.189,410.485,135.655,332.246,392.04,191.39,755.766,441.965,-126.804,821.315,407.066,-76.9979,750.776,396.233,-136.232,128.058,130.382,106.092,267.0

We will randomly add missing values to 15% of the data.  We will use the GLRM and Autoencoder methods to try to impute these missing values.

In [29]:
train_data = h2o.deep_copy(gait_data, 'train_gait.hex')

In [30]:
train_data.insert_missing_values(fraction = 0.15, seed = 1234)

Insert Missing Values progress: |█████████████████████████████████████████| 100%


Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
0,617.248,1055.28,170.782,639.606,1044.26,-88.9098,nan,1051.26,29.9668,517.333,741.096,nan,570.201,666.706,123.686,480.064,626.515,199.554,659.579,nan,-145.972,750.481,735.85,-73.4851,697.43,642.682,-137.37,360.904,474.04,186.849,420.867,412.366,134.719,343.737,388.037,nan,nan,446.177,-128.528,819.764,415.132,-81.7,750.046,397.941,-138.884,156.443,113.073,104.003,297.181,nan,67.0685,nan,73.7662,176.63,429.093,nan,nan,815.764,63.3979,nan,967.99,122.581,-32.7704,943.789,105.337,-122.344,1078.71,143.548,-60.9359,660.827,1353.78,20.1984,566.598,1449.58,229.997,596.424,nan,-186.7,593.641,1249.11,nan,580.735,1233.74,-222.927,567.003,1143.11,315.566,518.103,1146.32,-260.958,nan,973.779,273.788,746.207,944.378,nan,619.823,916.17,-218.327,574.163,905.877,-276.121,366.123,35.9082,193.141,375.866,39.8688,81.1719,1022.04,113.138,-136.458,1034.97,125.079,-34.9382,736.522,nan,nan,748.242,nan,-31.6672,nan,1747.61,46.7415
0.017,617.998,1053.22,nan,nan,1042.28,-90.9321,432.341,1050.24,26.8468,516.614,740.426,211.219,nan,664.199,123.394,479.64,625.653,199.315,661.613,nan,-146.439,756.852,736.423,-77.1608,nan,641.493,-135.753,357.694,nan,187.046,415.392,411.818,134.787,336.911,nan,190.88,755.121,nan,nan,820.42,409.886,-79.0317,nan,396.968,-137.308,141.984,121.291,105.038,281.979,91.6346,68.4662,nan,75.7492,177.511,409.884,43.1392,110.564,799.82,65.8006,-49.9463,957.452,nan,nan,943.789,105.337,nan,1069.28,nan,-56.2015,661.329,1352.71,17.2436,566.925,1448.94,226.133,nan,1466.41,nan,593.205,1248.56,276.044,582.396,1230.64,-225.945,566.137,1142.56,312.591,519.93,nan,nan,767.166,nan,272.553,742.406,941.011,nan,623.055,912.317,-220.419,576.286,902.639,-278.464,346.245,35.811,nan,nan,39.8057,80.9351,1011.08,101.084,-132.527,1024,111.614,-30.6797,736.042,1570.83,101.786,nan,1574.04,-36.6651,750.68,1746.67,41.1642
0.033,620.292,1051.77,165.859,643.597,1041.06,-94.3072,434.099,1049.34,nan,517.779,739.681,209.93,570.455,663.091,nan,nan,625.037,199.233,663.777,742.559,nan,760.839,737.405,-80.5061,702.586,640.568,-134.511,355.966,476.074,187.729,411.189,nan,135.655,332.246,392.04,191.39,755.766,441.965,-126.804,nan,407.066,-76.9979,750.776,396.233,nan,128.058,130.382,106.092,267.077,95.3412,69.8791,255.806,78.1227,178.353,390.768,43.1016,110.35,784.397,nan,nan,946.333,105.409,-23.486,932.771,97.3363,-120.643,nan,113.935,-52.2841,nan,1351.79,13.6311,567.57,1448.22,221.828,596.4

Our training data has 2,852 missing value entries that we will try to impute.

In [31]:
train_data.isna().sum()

2852.0

## Step 2. Imputation through GLRM

We will use the GLRM algorithm to impute the missing values. The GLRM algorithm is an extension of PCA and creates a linear, low dimensional representation of the data.  From the low dimensional data, we can reconstruct our frame and use it to impute any missing values.

For more details on the GLRM algorithm, see our GLRM tutorial: <https://github.com/h2oai/h2o-tutorials/blob/master/best-practices/glrm/README.md>

In [12]:
from h2o.estimators import H2OGeneralizedLowRankEstimator

glrm = H2OGeneralizedLowRankEstimator(k = 10, # reduce the dimensions of the original data to 10
                                      model_id = "glrm.hex")

glrm.train(training_frame = train_data, validation_frame = gait_data)

glrm Model Build progress: |██████████████████████████████████████████████| 100%


We can use the GLRM to reconstruct our training data. 

In [19]:
glrm_reconstruction = glrm.predict(train_data)

glrm prediction progress: |███████████████████████████████████████████████| 100%


In [20]:
glrm_reconstruction.head()

reconstr_Time,reconstr_R.ASIS.X,reconstr_R.ASIS.Y,reconstr_R.ASIS.Z,reconstr_L.ASIS.X,reconstr_L.ASIS.Y,reconstr_L.ASIS.Z,reconstr_V.Sacral.X,reconstr_V.Sacral.Y,reconstr_V.Sacral.Z,reconstr_R.Thigh.Upper.X,reconstr_R.Thigh.Upper.Y,reconstr_R.Thigh.Upper.Z,reconstr_R.Thigh.Front.X,reconstr_R.Thigh.Front.Y,reconstr_R.Thigh.Front.Z,reconstr_R.Thigh.Rear.X,reconstr_R.Thigh.Rear.Y,reconstr_R.Thigh.Rear.Z,reconstr_L.Thigh.Upper.X,reconstr_L.Thigh.Upper.Y,reconstr_L.Thigh.Upper.Z,reconstr_L.Thigh.Front.X,reconstr_L.Thigh.Front.Y,reconstr_L.Thigh.Front.Z,reconstr_L.Thigh.Rear.X,reconstr_L.Thigh.Rear.Y,reconstr_L.Thigh.Rear.Z,reconstr_R.Shank.Upper.X,reconstr_R.Shank.Upper.Y,reconstr_R.Shank.Upper.Z,reconstr_R.Shank.Front.X,reconstr_R.Shank.Front.Y,reconstr_R.Shank.Front.Z,reconstr_R.Shank.Rear.X,reconstr_R.Shank.Rear.Y,reconstr_R.Shank.Rear.Z,reconstr_L.Shank.Upper.X,reconstr_L.Shank.Upper.Y,reconstr_L.Shank.Upper.Z,reconstr_L.Shank.Front.X,reconstr_L.Shank.Front.Y,reconstr_L.Shank.Front.Z,reconstr_L.Shank.Rear.X,reconstr_L.Shank.Rear.Y,reconstr_L.Shank.Rear.Z,reconstr_R.Heel.X,reconstr_R.Heel.Y,reconstr_R.Heel.Z,reconstr_R.Midfoot.Sup.X,reconstr_R.Midfoot.Sup.Y,reconstr_R.Midfoot.Sup.Z,reconstr_R.Midfoot.Lat.X,reconstr_R.Midfoot.Lat.Y,reconstr_R.Midfoot.Lat.Z,reconstr_R.Toe.Tip.X,reconstr_R.Toe.Tip.Y,reconstr_R.Toe.Tip.Z,reconstr_L.Heel.X,reconstr_L.Heel.Y,reconstr_L.Heel.Z,reconstr_L.Midfoot.Sup.X,reconstr_L.Midfoot.Sup.Y,reconstr_L.Midfoot.Sup.Z,reconstr_L.Midfoot.Lat.X,reconstr_L.Midfoot.Lat.Y,reconstr_L.Midfoot.Lat.Z,reconstr_L.Toe.Tip.X,reconstr_L.Toe.Tip.Y,reconstr_L.Toe.Tip.Z,reconstr_Sternum.X,reconstr_Sternum.Y,reconstr_Sternum.Z,reconstr_R.Acromium.X,reconstr_R.Acromium.Y,reconstr_R.Acromium.Z,reconstr_L.Acromium.X,reconstr_L.Acromium.Y,reconstr_L.Acromium.Z,reconstr_R.Bicep.X,reconstr_R.Bicep.Y,reconstr_R.Bicep.Z,reconstr_L.Bicep.X,reconstr_L.Bicep.Y,reconstr_L.Bicep.Z,reconstr_R.Elbow.X,reconstr_R.Elbow.Y,reconstr_R.Elbow.Z,reconstr_L.Elbow.X,reconstr_L.Elbow.Y,reconstr_L.Elbow.Z,reconstr_R.Wrist.Med.X,reconstr_R.Wrist.Med.Y,reconstr_R.Wrist.Med.Z,reconstr_R.Wrist.Lat.X,reconstr_R.Wrist.Lat.Y,reconstr_R.Wrist.Lat.Z,reconstr_L.Wrist.Med.X,reconstr_L.Wrist.Med.Y,reconstr_L.Wrist.Med.Z,reconstr_L.Wrist.Lat.X,reconstr_L.Wrist.Lat.Y,reconstr_L.Wrist.Lat.Z,reconstr_R.Toe.Lat.X,reconstr_R.Toe.Lat.Y,reconstr_R.Toe.Lat.Z,reconstr_R.Toe.Med.X,reconstr_R.Toe.Med.Y,reconstr_R.Toe.Med.Z,reconstr_L.Toe.Lat.X,reconstr_L.Toe.Lat.Y,reconstr_L.Toe.Lat.Z,reconstr_L.Toe.Med.X,reconstr_L.Toe.Med.Y,reconstr_L.Toe.Med.Z,reconstr_R.Temple.X,reconstr_R.Temple.Y,reconstr_R.Temple.Z,reconstr_L.Temple.X,reconstr_L.Temple.Y,reconstr_L.Temple.Z,reconstr_Top.Head.X,reconstr_Top.Head.Y,reconstr_Top.Head.Z
1.20217,614.078,1064.69,143.979,622.791,1052.55,-114.613,420.728,1058.15,19.3402,607.987,751.656,195.966,682.023,697.2,106.334,618.93,632.527,189.074,581.383,756.189,-167.041,673.255,726.405,-106.616,580.483,650.09,-151.726,560.179,476.773,177.782,615.192,429.166,127.831,556.906,398.166,183.767,563.574,450.723,-146.723,597.942,396.766,-94.1588,537.855,414.842,-151.866,418.856,133.825,90.8245,549.007,115.666,70.0718,546.65,102.333,174.585,666.612,70.2616,121.041,421.251,124.919,-51.3747,581.714,100.503,-29.5848,564.885,93.0878,-122.706,661.719,65.7446,-55.3951,645.495,1367.34,5.80562,541.827,1459.29,212.494,582.894,1477.43,-205.55,543.082,1258.85,260.371,600.553,1242.8,-233.845,501.435,1157.66,286.781,554.648,1145.68,-271.617,650.74,938.207,280.747,613.809,923.56,329.812,686.311,939.361,-214.594,644.903,920.179,-274.43,609.256,65.1775,194.723,635.447,71.8467,87.3352,618.06,64.1636,-132.331,647.025,70.4461,-31.1238,707.395,1595.83,90.1739,719.606,1600.98,-46.9489,700.679,1769.91,30.3159
1.20223,614.078,1064.69,143.979,622.79,1052.55,-114.612,420.727,1058.15,19.3418,607.987,751.656,195.967,682.023,697.2,106.333,618.93,632.526,189.074,581.381,756.189,-167.04,673.254,726.405,-106.616,580.481,650.09,-151.726,560.182,476.771,177.782,615.197,429.166,127.831,5

Our reconstructed data has no missing values.

In [21]:
glrm_reconstruction.isna().sum()

0.0

We will now calculate the sum of squared errors on the missing values.  This will help us determine how well the GLRM model does at imputing our missing values.

In [69]:
glrm_performance = glrm.model_performance(valid = True)

print("Sum of Squared Error: " + "{:,.0f}".format(glrm_performance.num_err()))
print("Mean of Squared Error: " + "{:,.0f}".format(glrm_performance.num_err()/train_data.isna().sum()))

Sum of Squared Error: 135,970,721
Mean of Squared Error: 47,676


We can try lowering the squared error on the missing value entries by tuning the parameters of the GLRM model: https://github.com/h2oai/h2o-tutorials/blob/master/best-practices/glrm/README.md#determining-best-parameters

## Step 3. Imputation through Autoencoder

Section coming soon.